In [16]:

# Instalações necessárias das apis do google para rodar este projeto

%pip -q install google-genai
!pip install -q google-adk

In [17]:
# Configurar a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [18]:
# Configurar o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [19]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# 🧪 Carboninho - Assistente Virtual de Química
print("🧬 Oi! Sou o Carboninho, seu assistente virtual de laboratório químico! 👨‍🔬🔬")
print("\n")

# --- Funções auxiliares ---
from IPython.display import Markdown
import textwrap

def to_markdown(text):
    text = text.replace('•', '  *')
    text = text.replace("## ", "### ")
    text = text.replace("# ", "## ")
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent, message_text):
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return f"## 🧪 Resultado do Carboninho\n\n{final_response.strip()}"

# --- Definição dos agentes ---

def agente_busca_quimica(pergunta):
    agente = Agent(
        name="agente_busca_quimica",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em química. Use o google_search para buscar informações detalhadas e precisas sobre:
        '{pergunta}'. Foque em propriedades, reações, mecanismos, aplicações, etc.
        """,
        description="Agente que busca informações químicas no Google",
        tools=[google_search]
    )
    return call_agent(agente, f"🔍 Buscar informações sobre: {pergunta}")

def agente_explica_conceito(conceito):
    agente = Agent(
        name="agente_explica_conceito",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um professor de química. Explique o conceito '{conceito}' de forma clara, com exemplos do cotidiano se possível.
        """,
        description="Agente explicador de conceitos químicos",
        tools=[]
    )
    return call_agent(agente, f"📘 Explicar conceito: {conceito}")

def agente_planeja_experimento_virtual(objetivo, nivel):
    agente = Agent(
        name="agente_planeja_experimento_virtual",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em laboratório virtual. Ajude um estudante de nível '{nivel}' a planejar um experimento virtual
        com objetivo '{objetivo}'. Sugira simuladores, passo a passo e cuidados de segurança.
        """,
        description="Planejador de experimentos virtuais de química",
        tools=[google_search]
    )
    return call_agent(agente, f"🧪 Planejar experimento: objetivo={objetivo}, nível={nivel}")

def agente_calcula_quimica():
    return Agent(
        name="agente_calcula_quimica",
        model="gemini-2.0-flash",
        instruction="""
        Você resolve cálculos químicos: molaridade, pH, estequiometria, etc. Gere código Python comentado para resolver.
        Peça mais dados se necessário.
        """,
        description="Calculadora de química com código Python",
        tools=[]
    )

def agente_seguranca_laboratorio(pergunta, composto=None, procedimento=None):
    agente = Agent(
        name="agente_seguranca_laboratorio",
        model="gemini-2.0-flash",
        instruction=f"""
        Especialista em segurança de laboratório. Responda à pergunta '{pergunta}' com foco em segurança.
        Inclua informações sobre '{composto}' e '{procedimento}' se fornecidos.
        Liste EPIs, riscos e ações de emergência.
        """,
        description="Agente de segurança de laboratório químico",
        tools=[google_search]
    )
    entrada = f"🔐 {pergunta}"
    if composto:
        entrada += f"\n🧪 Composto: {composto}"
    if procedimento:
        entrada += f"\n🧫 Procedimento: {procedimento}"
    return call_agent(agente, entrada)

def agente_resumo_final(nome_usuario, interacoes):
    agente = Agent(
        name="agente_resumo_final",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente educacional que vai gerar um resumo final da sessão para o usuário '{nome_usuario}'.
        Com base nas interações registradas abaixo, gere um texto que:
        - Recapitule os principais tópicos abordados;
        - Destaque aprendizados ou respostas recebidas;
        - Sugira próximos passos ou tópicos relacionados para estudo futuro.
        Utilize emojis educativos e estrutura de tópicos.
        """,
        description="Resumo final da sessão com dicas e revisão de conteúdo",
        tools=[]
    )
    entrada = f"📘 Interações da sessão:\n{interacoes}"
    return call_agent(agente, entrada)

def calcular(agente, pergunta):
    return call_agent(agente, f"🧮 Pergunta de cálculo: {pergunta}")

# --- Fluxo principal ---

def run_carboninho():
    nome = input("Informe seu nome: ").strip()
    interacoes_usuario = ""

    while True:
        print()  # linha em branco para espaçar
        nivel = input("📊 Nível de conhecimento em química (iniciante/médio/avançado) ou 'sair': ").lower().strip()
        if nivel == "sair":
            print(f"\n🔬 Encerramento: Foi ótimo conversar com você, {nome}! Até a próxima! 👋")
            break

        objetivo = input("\n🎯 Qual o seu objetivo principal hoje? ").strip()

        while True:
            pergunta = input("\n💬 Faça sua pergunta sobre química (ou digite 'sair' para encerrar): ").strip()
            if pergunta.lower() == "sair":
                print(f"\n📚 Resumo da sessão com {nome}:\n")
                resumo = agente_resumo_final(nome, interacoes_usuario)
                display(to_markdown(resumo))
                print(f"\n🔬 Foi ótimo conversar com você, {nome}! Até a próxima! 👋")
                return

            p = pergunta.lower()
            if any(x in p for x in ["calcular", "cálculo", "molaridade", "estequiometria", "ph"]):
                resposta = calcular(agente_calcula_quimica(), pergunta)
            elif any(x in p for x in ["segurança", "epi", "risco", "precaução", "emergência"]):
                resposta = agente_seguranca_laboratorio(pergunta)
            elif any(x in p for x in ["experimento", "laboratório virtual", "simulação", "planejar experimento"]):
                resposta = agente_planeja_experimento_virtual(objetivo, nivel)
            elif any(x in p for x in ["explicar", "conceito", "definição", "o que é", "como funciona"]):
                resposta = agente_explica_conceito(pergunta)
            else:
                resposta = agente_busca_quimica(pergunta)

            interacoes_usuario += f"\n🔎 Pergunta: {pergunta}\n📥 Resposta: {resposta}\n"
            display(to_markdown(resposta))

# --- Execução ---
if __name__ == "__main__":
    run_carboninho()